# Kryptologie Lab - Übung 04

In [1]:
import numpy as np

In [2]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[0xE,0xB,0xD,9],[9,0xE,0xB,0xD],[0xD,9,0xE,0xB],[0xB,0xD,9,0xE]])

In [3]:
with open('SBox.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))

with open('SBoxInvers.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox_inverse = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))

## Init Galoiskörper-Operationen

In [4]:
def gfAddition(a,b):
    return np.bitwise_xor(a,b)

def xTimes(a):
    t = a << 1
    if a & (1 << 7) != 0:
        t = t ^ 0x1b
    return t

def gfMultiply(a,b):
    index = 0
    out = 0
    while b != 0:
        if (b & 1 == 1):
            val = a
            for i in range(index):
                val = xTimes(val)
            out = gfAddition(out, val)
        index += 1
        b = b >> 1
    return out

## Init algorithm steps

In [5]:
def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    if (len(chars) < 16):
        fill = [0 for i in range(16 - len(chars))]
        chars.extend(fill)
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4), order="F")

def blockToText(textblock):
    text = textblock.transpose().flatten()
    toChar = lambda c: chr(c)
    return "".join(np.array([toChar(c) for c in text]))

def addRoundKey(textblock, key):
    return np.bitwise_xor(textblock, key)
    
def subByte(byte, box):
    row = (byte >> 4) & 0xF
    col = byte & 0xF
    return box[row, col]

def subBytes(textblock, reverse=False):
    if reverse:
        box = sbox_inverse
    else:
        box = sbox
    for y in range(len(textblock)):
        for x in range(len(textblock[y])):
            textblock[y,x] = subByte(textblock[y,x], box)
    return textblock

def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMultiply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[:, column] = subColumn(textblock[:, column], matrix)
    return textblock

## Init algorithm process

In [6]:
def encryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    return blockToText(textblock)

def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[10])
    textblock = shiftRows(textblock, reverse=True)
    textblock = subBytes(textblock, reverse=True)
    for i in range(9, 0, -1):
        textblock = addRoundKey(textblock, keys[i])
        textblock = mixColumns(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = subBytes(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[0])
    return blockToText(textblock)

## Init block splitting

In [7]:
def encrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

def decrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

## Schlüsselgenerierung

In [8]:
key = ["EINSCHLUESSEL123" for i in range(11)]

In [22]:
def rotWord(word):
    return np.roll(word, shift=-1)

def subWord(word):
    return [subByte(byte, sbox) for byte in word]

def rcon(i):
    array = [0x1, 0x2, 0x4, 0x8, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]
    return [array[i], 0,0,0]

# input 16 character string
def generateKey(key):
    block = initBlock(key)
    words = []
    words.append(block[:,0])
    words.append(block[:,1])
    words.append(block[:,2])
    words.append(block[:,3])
    for i in range(4, 44):
        if i % 4 == 0:
            val = gfAddition(words[i-4], rcon(i // 4 - 1))
            val = gfAddition(subWord(rotWord(words[i-1])), val)
            words.append(val)
        else:
            words.append(gfAddition(words[i-4], words[i-1]))
    keys = np.empty((11, 4,4), dtype=np.dtype('B'))
    for i in range(0, 44):
        keys[i // 4, :, i % 4] = words[i]
    return keys

In [23]:
keys = generateKey("EINSCHLUESSEL123")
print(keys)
print(keys.shape)

[[[ 69  67  69  76]
  [ 73  72  83  49]
  [ 78  76  83  50]
  [ 83  85  69  51]]

 [[131 192 133 201]
  [106  34 113  64]
  [141 193 146 160]
  [122  47 106  89]]

 [[136  72 205   4]
  [138 168 217 153]
  [ 70 135  21 181]
  [167 136 226 187]]

 [[ 98  42 231 227]
  [ 95 247  46 183]
  [172  43  62 139]
  [ 85 221  63 132]]

 [[195 233  14 237]
  [ 98 149 187  12]
  [243 216 230 109]
  [ 68 153 166  34]]

 [[ 45 196 202  39]
  [ 94 203 112 124]
  [ 96 184  94  51]
  [ 17 136  46  12]]

 [[ 29 217  19  52]
  [157  86  38  90]
  [158  38 120  75]
  [221  85 123 119]]

 [[227  58  41  29]
  [ 46 120  94   4]
  [107  77  53 126]
  [197 144 235 156]]

 [[145 171 130 159]
  [221 165 251 255]
  [181 248 205 179]
  [ 97 241  26 134]]

 [[156  55 181  42]
  [176  21 238  17]
  [241   9 196 119]
  [186  75  81 215]]

 [[ 40  31 170 128]
  [ 69  80 190 175]
  [255 246  50  69]
  [ 95  20  69 146]]]
(11, 4, 4)


In [ ]:
text = "Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger."
print("Input: ", text)
cipher = encrypt(text, keys)
print("cypher: ", cipher)
new_text = decrypt(cipher, keys)
print("text: ", new_text)

Input:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.
cypher:  OFÞè2ï¸£öãYèÖ\þ±ètî@jkà¤G5aFúýÆÄf·mÌÖrO°xùcNuV>yîÂ¹oRyËYþ[!Azæk ÒoH»ó+Ìþ×äµâbJ¶
text:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.
